In [76]:
import numpy as np
import scipy.stats

data = np.array([55, 10, 2, 6, 4, 10, 2, 5, 6])


def Aiken_Alpha(contingency_table, confidence_level):
    
    y = np.reshape(contingency_table, (int(np.sqrt(len(contingency_table))), -1))

    # Calculation of Cohen's Kappa
    Weights_matrix = np.eye(y.shape[0])
    confidence_Level = 0.95
    Number_Of_Levels = y.shape[0]
    y = y + 1 / (Number_Of_Levels**2)
    Sample_Size = np.sum(y)
    Weighted_Sum = np.sum(y * Weights_matrix)
    Observed_Agreement = Weighted_Sum / Sample_Size
    Sum_Of_Rows = np.sum(y, axis=1)
    Sum_Of_Coloumns = np.sum(y, axis=0)
    Probability_Rows = Sum_Of_Rows / Sample_Size
    Probability_Coloumns = Sum_Of_Coloumns / Sample_Size
    Expected_Agreement = np.sum(np.tile(Probability_Coloumns, (Number_Of_Levels, 1)) * Probability_Rows.reshape(-1, 1) * Weights_matrix)
    alpha = (Observed_Agreement - Expected_Agreement) / (1 - Expected_Agreement)

    # Calculation of Aiken's Alpha
    continue_loop = True
    while continue_loop:
        previous_alpha = alpha
        pr_denominator = Sample_Size * (1 - alpha + alpha * np.dot(Weights_matrix, Probability_Coloumns) / Expected_Agreement)
        Probability_Rows = Sum_Of_Rows / pr_denominator
        Probability_Rows[0] = 1 - np.sum(Probability_Rows[1:])
        pc_denominator = Sample_Size * (1 - alpha + alpha * np.dot(Probability_Rows, Weights_matrix) / Expected_Agreement)
        Probability_Coloumns = Sum_Of_Coloumns / pc_denominator
        Probability_Coloumns[0] = 1 - np.sum(Probability_Coloumns[1:])
        Expected_Agreement = np.sum(np.tile(Probability_Coloumns, (Number_Of_Levels, 1)) * Probability_Rows.reshape(-1, 1) * Weights_matrix)
        alpha = (Observed_Agreement - Expected_Agreement) / (1 - Expected_Agreement)
        continue_loop = abs(alpha - previous_alpha) > 0.00001

    Aikens_Alpha = alpha

    # Calculation of the Standard Deviation
    Proababilty_Difference_Rows = Probability_Rows[1:] - Probability_Rows[0]
    Proababilty_Difference_Coloumns = Probability_Coloumns[1:] - Probability_Coloumns[0]

    V = alpha / Expected_Agreement / ((1 - Aikens_Alpha) * Expected_Agreement + Aikens_Alpha)
    T = 1 / Aikens_Alpha - 1
    First_Second_Derivatives = -Sample_Size * (1 - Expected_Agreement) / (1 - Aikens_Alpha) / ((1 - Aikens_Alpha) * Expected_Agreement + Aikens_Alpha)
    First_Second_Derivatives_Coloumns = -Weighted_Sum * Proababilty_Difference_Coloumns * ((Sample_Size / Weighted_Sum) ** 2)
    First_Second_Derivatives_Rows = -Weighted_Sum * Proababilty_Difference_Rows * ((Sample_Size / Weighted_Sum) ** 2)

    Rows_Matrix = -np.sum(y[:, 0]) / (Probability_Coloumns[0] ** 2) + Weighted_Sum * (2 * Expected_Agreement * T + 1) * V**2 * np.outer(Proababilty_Difference_Rows, Proababilty_Difference_Rows)
    Rows_Term = -Sum_Of_Coloumns[1:] / (Probability_Coloumns[1:] ** 2) - Sum_Of_Coloumns[0] / (Probability_Coloumns[0] ** 2) + Weighted_Sum * (2 * Expected_Agreement * T + 1) * V **2 * (Proababilty_Difference_Rows ** 2)
    np.fill_diagonal(Rows_Matrix, Rows_Term)

    Coloumns_Matrix = -np.sum(y[0, :]) / (Probability_Rows[0] ** 2) + Weighted_Sum * (2 * Expected_Agreement * T + 1) * V**2 * np.outer(Proababilty_Difference_Coloumns, Proababilty_Difference_Coloumns)
    Cols_Term = -Sum_Of_Rows[1:] / (Probability_Rows[1:] ** 2) - Sum_Of_Rows[0] / (Probability_Rows[0] ** 2) + Weighted_Sum * (2 * Expected_Agreement * T + 1) * V**2 * (Proababilty_Difference_Coloumns ** 2)
    np.fill_diagonal(Coloumns_Matrix, Cols_Term)

    Rows_Coloumns_Matrix = -Weighted_Sum * V + Weighted_Sum * (2 * Expected_Agreement * T + 1) * V**2 * np.outer(Proababilty_Difference_Coloumns, Proababilty_Difference_Rows).T
    Rows_Cols_Term = -2 * Weighted_Sum * V + Weighted_Sum * (2 * Expected_Agreement * T + 1) * V **2 * Proababilty_Difference_Rows * Proababilty_Difference_Coloumns
    np.fill_diagonal(Rows_Coloumns_Matrix, Rows_Cols_Term)

    Var_Matrix_Top = np.concatenate(([First_Second_Derivatives], First_Second_Derivatives_Rows, First_Second_Derivatives_Coloumns))
    Var_Matrix_Center = np.column_stack((np.matrix(First_Second_Derivatives_Rows).T, Rows_Matrix, Rows_Coloumns_Matrix))
    Var_Matrix_Bottom = np.column_stack((np.matrix(First_Second_Derivatives_Coloumns).T, Rows_Coloumns_Matrix.T, Coloumns_Matrix))
    Var_Matrix_Final = np.vstack((Var_Matrix_Top, Var_Matrix_Center, Var_Matrix_Bottom))
    Covariance_Matrix = np.linalg.inv(-Var_Matrix_Final)

    Aikens_Standart_Deviation = np.sqrt(Covariance_Matrix[0, 0])

    # Confidence Interval
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    LowerCi = Aikens_Alpha - zcrit * Aikens_Standart_Deviation
    UpperCi = Aikens_Alpha + zcrit * Aikens_Standart_Deviation


    results= {}

    results["Aiken's Alpha"]= Aikens_Alpha
    results["Aiken's Standard Deviation "]= Aikens_Standart_Deviation
    #results["Statistic Z"] = Z_statistic
    #results["p_value"] = p_value
    results["Confidence Intervals"] = f"({round(LowerCi, 4)}, {round(UpperCi, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return results

Aiken_Alpha(data, 0.95)






{"Aiken's Alpha": 0.4006062801486114,
 "Aiken's Standard Deviation ": 0.08468763496148539,
 'Confidence Intervals': '(0.2346, 0.5666)'}